In [1]:
!ls ../input/abide-cc200/cc200/

CC200


In [2]:
#options: cc200, dosenbach160, aal
p_ROI = "cc200"
p_fold = 10
p_center = "Stanford"
p_mode = "whole"
p_augmentation = True
p_Method = "ASD-DiagNet"

In [3]:
parameter_list = [p_ROI,p_fold,p_center,p_mode,p_augmentation,p_Method]
print("*****List of patameters****")
print("ROI atlas: ",p_ROI)
print("per Center or whole: ",p_mode)
if p_mode == 'percenter':
    print("Center's name: ",p_center)
print("Method's name: ",p_Method)
if p_Method == "ASD-DiagNet":
    print("Augmentation: ",p_augmentation)


*****List of patameters****
ROI atlas:  cc200
per Center or whole:  whole
Method's name:  ASD-DiagNet
Augmentation:  True


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import pyprind
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import silhouette_samples, silhouette_score
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn import tree
from sklearn.cluster import MiniBatchKMeans
import functools
import numpy.ma as ma # for masked arrays
import pyprind
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm
import wandb
from itertools import groupby


# !wandb login d164742a4a99e4e581f543102aff0992153ad225

## Importing the data 

In [5]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3]) 
    else:
        key = '_'.join(f_split[0:2])
    return key

In [6]:
cc200_data_path = '../input/abide-cc200/cc200/CC200'#cc200'#path to time series data
flist = os.listdir(cc200_data_path)

fid = []
for f in flist:
    fid.append(get_key(f))

    
data_df = pd.read_csv('../input/abide-cc200/Phenotypic_V1_0b_preprocessed1.csv')#path 
data_df = data_df[data_df['FILE_ID'].isin(fid)]
data_df.DX_GROUP = data_df.DX_GROUP.map({1: 1, 2:0})
data_df['FILE_PATH'] = data_df['FILE_ID'].apply(lambda x : os.path.join(cc200_data_path,x + '_rois_cc200.1D')) 


# print(data_df.head())
print(len(data_df))
print(data_df['FILE_PATH'].values[0])

1035
../input/abide-cc200/cc200/CC200/Pitt_0050003_rois_cc200.1D


In [7]:
len(flist)

1035

### Helper functions for computing correlations

In [8]:
def get_label(filename):
    assert (filename in labels)
    return labels[filename]


def get_corr_data(path):
    
    df = pd.read_csv(path, sep='\t')
            
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()


def get_corr_matrix(filename,data_path):
    # returns correlation matrix
    for file in os.listdir(data_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

def get_regs(samplesnames,regnum):
    # returns region index array
    datas = []
    for sn in samplesnames:
        datas.append(all_corr[sn][0])
    datas = np.array(datas)     # datas shape len(samplesnames)x19900
    avg=[]
    for ie in range(datas.shape[1]):
        avg.append(np.mean(datas[:,ie]))
    avg=np.array(avg)
    highs=avg.argsort()[-regnum:][::-1]
    lows=avg.argsort()[:regnum][::-1]
    regions=np.concatenate((highs,lows),axis=0) # shape (9950,)
    return regions

In [9]:
labels = data_df['DX_GROUP'].values
fpaths = data_df['FILE_PATH']
sfc = []


for path in fpaths:
    corr_data = get_corr_data(path)
    sfc.append(corr_data)
    
sfc = np.asarray(sfc)
print(sfc.shape)
print(labels.shape)

(1035, 19900)
(1035,)


In [10]:
print(np.min(sfc))

print(np.max(sfc))

-0.9592505245533961
0.9900251023663668


## Defining dataset class

In [11]:
class ASDDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        pass
    def __getitem__(self,idx):
        return torch.tensor(self.x[idx],dtype=torch.float),torch.tensor(self.y[idx],dtype=torch.float)
        pass
    def __len__(self):
        return len(self.x)
        pass
        

In [12]:
d = ASDDataset(sfc,labels)

d.__getitem__(0)

(tensor([ 0.2489,  0.1380,  0.1472,  ...,  0.0305,  0.1095, -0.2725]),
 tensor(1.))

## Defining Autoencoder class

In [13]:
class MTAutoEncoder(nn.Module):
    def __init__(self, num_inputs=990, 
                 num_latent=200, tied=True,
                 num_classes=2, use_dropout=False):
        super(MTAutoEncoder, self).__init__()
        
        self.num_latent = num_latent
        self.num_inputs = num_inputs
        
        self.fc_encoder = nn.Sequential (
                nn.Linear(self.num_inputs,4096),
                nn.Tanh(),
                nn.Linear(4096,256),
                nn.Tanh())
        
        self.fc_decoder = nn.Sequential (
                nn.Linear(256,4096),
                nn.Tanh(),
                nn.Linear(4096,self.num_inputs),
                nn.Tanh())
         
        
        if use_dropout:
            self.classifier = nn.Sequential (
                nn.Dropout(p=0.25),
                nn.Linear(256, 1),
#                 nn.Sigmoid(),
#                 nn.Linear(128, 1),

            )
        else:
            self.classifier = nn.Sequential (
                nn.Linear(256, 1),
#                 nn.Sigmoid(),
#                 nn.Linear(128, 1),
            )
            
         
    def forward(self, x, eval_classifier=False):

        if eval_classifier:
            x = self.fc_encoder(x)
            x_logit = self.classifier(x).squeeze()
            
        else:
            x = self.fc_encoder(x)
            x = self.fc_decoder(x)
            x_logit = None
            
        return x, x_logit

mtae = MTAutoEncoder()

mtae

MTAutoEncoder(
  (fc_encoder): Sequential(
    (0): Linear(in_features=990, out_features=4096, bias=True)
    (1): Tanh()
    (2): Linear(in_features=4096, out_features=256, bias=True)
    (3): Tanh()
  )
  (fc_decoder): Sequential(
    (0): Linear(in_features=256, out_features=4096, bias=True)
    (1): Tanh()
    (2): Linear(in_features=4096, out_features=990, bias=True)
    (3): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=1, bias=True)
  )
)

## Defining training and testing functions

In [14]:
def train(model, epoch, train_loader, p_bernoulli=None, mode='both', lam_factor=1.0):
    model.train()
    train_losses = []
    clf_train_loss = []
    ae_train_loss = []
    
    if mode == 'clf':
        final_targets = []
        final_predictions = []
    else:
        final_targets = None
        final_predictions = None    
    
    for i,(batch_x,batch_y) in enumerate(train_loader):
        if len(batch_x) != batch_size:
            continue
        if p_bernoulli is not None:
            if i == 0:
                p_tensor = torch.ones_like(batch_x).to(device)*p_bernoulli
            rand_bernoulli = torch.bernoulli(p_tensor).to(device)

        data, target = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()

        if mode == 'ae':
            if p_bernoulli is not None:
                rec_noisy, _ = model(data*rand_bernoulli, False)
                loss_ae = criterion_ae(rec_noisy, data) / len(batch_x)
            else:
                rec, _ = model(data, False)
                loss_ae = criterion_ae(rec, data) / len(batch_x)
                
            loss_total = loss_ae
            loss_ae_np = loss_ae.detach().cpu().numpy()
            
            clf_train_loss.append(0.0)
            ae_train_loss.append(loss_ae_np)
            train_losses.append([loss_ae_np, 0.0])
                
        if mode == 'clf':
            rec_clean, logits = model(data, True)
            loss_clf = criterion_clf(logits, target)
            
            proba = torch.sigmoid(logits).detach().cpu().numpy()
            predictions = np.ones_like(proba, dtype=np.int32)
            predictions[proba < 0.5] = 0
            
            final_targets.append(target.detach().cpu().numpy())
            final_predictions.append(predictions)
            
            loss_total = loss_clf
            loss_clf_np = loss_clf.detach().cpu().numpy()
            
            clf_train_loss.append(loss_clf_np)
            ae_train_loss.append(0.0)
            train_losses.append([0.0,loss_clf_np])
            

        loss_total.backward()
        optimizer.step()
    
    if (final_targets is not None) and (final_predictions is not None):
        final_targets = np.concatenate(final_targets)
        final_predictions = np.concatenate(final_predictions)
        train_accuracy = np.mean(final_targets == final_predictions)

        return np.mean(clf_train_loss), train_accuracy
    else:
        return np.mean(ae_train_loss), None

def test(model, criterion, test_loader, 
         eval_classifier=False, num_batch=None):
    test_loss, n_test, correct = 0.0, 0, 0
    eval_loss = []
    all_predss=[]
    if eval_classifier:
        y_true, y_pred = [], []
    with torch.no_grad():
        model.eval()
        for i,(batch_x,batch_y) in enumerate(test_loader):
            if num_batch is not None:
                if i >= num_batch:
                    continue
            data, target = batch_x.to(device), batch_y.to(device)
            rec, logits = model(data, eval_classifier)

#             test_loss += criterion(rec, data).detach().cpu().numpy() 
#             n_test += len(batch_x)
            test_loss = criterion(logits, target).detach().cpu().numpy() 
            eval_loss.append(test_loss)
            if eval_classifier:
                proba = torch.sigmoid(logits).detach().cpu().numpy()
                preds = np.ones_like(proba, dtype=np.int32)
                preds[proba < 0.5] = 0
                all_predss.extend(preds)###????
                y_arr = np.array(batch_y, dtype=np.int32)

                correct += np.sum(preds == y_arr)
                y_true.extend(y_arr.tolist())
                y_pred.extend(proba.tolist())
        mlp_acc,mlp_sens,mlp_spef = confusion(y_true,all_predss)
        metrics_dict = {'accuracy': np.round(mlp_acc, 4), 
                        'senstivity' : np.round(mlp_sens,4), 
                        'specificity' : np.round(mlp_spef,4), 
                        'loss' : np.round(np.mean(eval_loss),4)}
        
    return  metrics_dict

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [17]:
if p_Method == "ASD-DiagNet" and p_mode == "whole":
    
    start = time.time()
    batch_size = 16
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
    num_epochs = 50

    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = True

    start= time.time()

#     print('p_bernoulli: ', p_bernoulli)
#     print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
#           'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
#     print('use_dropout: ', use_dropout, '\n')

    kk=0
    
    # list to store metrics after each fold
    repeat_acc=[]
    repeat_sen=[]
    repeat_spec=[]
    repeat_loss=[]
    
    
    for rp in range(1):
        kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
    # list to store metrics after each fold
        crossval_acc=[]
        crossval_sen=[]
        crossval_spec=[]
        crossval_loss=[]
        
        for kk,(train_index, test_index) in enumerate(kf.split(sfc, labels)):
            
            NAME = f'asd-diagnet-fold-{kk+1}-rp-2'
            ID = f'fold-{kk+1}-rp-2.1'
        
            x_train, y_train = sfc[train_index],labels[train_index]
            x_test, y_test = sfc[test_index],labels[test_index]


            verbose = (True if (kk == 0) else False)



            num_inpp = 19900
            n_lat = 512
            
            train_dataset = ASDDataset(x_train,y_train)
            test_dataset = ASDDataset(x_test,y_test)

            train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
            test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
               

            model = MTAutoEncoder(tied=False, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
            model.to(device)

            criterion_ae = nn.MSELoss(reduction='sum')
            criterion_clf = nn.BCEWithLogitsLoss()
#             optimizer = optim.SGD([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
#                                    {'params': model.fc_decoder.parameters(), 'lr': learning_rate_ae},
#                                    {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
#                                   momentum=0.9)
            
            optimizer = optim.Adam(model.parameters(),lr=0.0001,weight_decay=0.1)          



            for epoch in range(1, num_epochs+1):
                
                if epoch <= 30:
                    ae_train_loss,_ = train(model, epoch, train_dataloader, p_bernoulli, mode='ae')
                    content = f'AE Train loss: {(ae_train_loss):.4f}'

                else:
                    clf_train_loss, train_acc = train(model, epoch, train_dataloader, p_bernoulli, mode='clf')
                    content = f'CLF Train loss: {(clf_train_loss):.4f}, Train Accuracy: {(train_acc):.4f}'
                    
                print(f'Epoch {epoch}/{num_epochs+1}')
                print(content)

            metrics_dict = test(model, criterion_clf, test_dataloader, eval_classifier=True)
            print("-----------------------------")
            print(f'Fold {kk+1}/{p_fold}')
            content = f'{metrics_dict}'
            print(content)
            print("-----------------------------")
            
            crossval_acc.append(metrics_dict['accuracy'])
            crossval_sen.append(metrics_dict['senstivity'])
            crossval_spec.append(metrics_dict['specificity'])
            crossval_loss.append(metrics_dict['loss'])
            
            #save the model after each fold
            
            recorder = {'optimizer': optimizer.state_dict(),
            'model': model.state_dict(),
            'fold' : kk+1,
            'repitition' : rp+1}

            torch.save(recorder, f'{NAME}.pt')
            
        print("*********************************")    
        print(f'Average Value after 10 Folds and repeats one {rp+1}------->')
        content = f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}'
        print(content)
        print("*********************************") 
        
        repeat_acc.append(np.mean(crossval_acc))
        repeat_sen.append(np.mean(crossval_sen))
        repeat_spec.append(np.mean(crossval_spec))
        repeat_loss.append(np.mean(crossval_loss))
    
    print(f"Average Value after 1 Repeat:")
    content = f'Accuracy: {np.round(np.mean(repeat_acc),4)}, Senstivity: {np.round(np.mean(repeat_sen),4)}, Specificity: {np.round(np.mean(repeat_spec),4)}, Loss: {np.round(np.mean(repeat_loss),4)}'
    print(content)
        
    finish= time.time()
    print(finish-start)




Epoch 1/51
AE Train loss: 780.3627
Epoch 2/51
AE Train loss: 641.8359
Epoch 3/51
AE Train loss: 574.0011
Epoch 4/51
AE Train loss: 521.6058
Epoch 5/51
AE Train loss: 480.1600
Epoch 6/51
AE Train loss: 445.4202
Epoch 7/51
AE Train loss: 416.6529
Epoch 8/51
AE Train loss: 393.2597
Epoch 9/51
AE Train loss: 374.3742
Epoch 10/51
AE Train loss: 358.5388
Epoch 11/51
AE Train loss: 346.5001
Epoch 12/51
AE Train loss: 337.1112
Epoch 13/51
AE Train loss: 330.1006
Epoch 14/51
AE Train loss: 323.8033
Epoch 15/51
AE Train loss: 318.3643
Epoch 16/51
AE Train loss: 314.7458
Epoch 17/51
AE Train loss: 312.3139
Epoch 18/51
AE Train loss: 311.2360
Epoch 19/51
AE Train loss: 310.5615
Epoch 20/51
AE Train loss: 309.4768
Epoch 21/51
AE Train loss: 308.4588
Epoch 22/51
AE Train loss: 308.0207
Epoch 23/51
AE Train loss: 307.2036
Epoch 24/51
AE Train loss: 307.1804
Epoch 25/51
AE Train loss: 306.6443
Epoch 26/51
AE Train loss: 305.9350
Epoch 27/51
AE Train loss: 305.1592
Epoch 28/51
AE Train loss: 303.4073
E

In [ ]:
Accuracy: 0.6724, Senstivity: 0.61, Specificity: 0.7321, Loss: 0.6234999895095825
589.3430550098419

In [ ]:
adam with weight decay 0.3 

Accuracy: 0.657, Senstivity: 0.5431, Specificity: 0.766, Loss: 0.6509000062942505
1160.326141834259


adam w/o weight decay 
Accuracy: 0.7043, Senstivity: 0.6636, Specificity: 0.7434, Loss: 0.5964999794960022
1056.0533220767975


adam with weight decay 0.1 
Average Value after 1 Repeat:
Accuracy: 0.7063, Senstivity: 0.6716, Specificity: 0.7396, Loss: 0.5821999907493591
1162.2546133995056

adam with weight decay 0.15 
Accuracy: 0.686, Senstivity: 0.6598, Specificity: 0.7113, Loss: 0.5911999940872192
1161.7236032485962

adam with weight decay 0.1  and dropout 0.25
Accuracy: 0.6985, Senstivity: 0.6793, Specificity: 0.717, Loss: 0.5842000246047974
1162.4763264656067